## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,2022-09-06 04:37:42,-22.9661,-42.0278,66.25,65,88,11.99,overcast clouds
1,1,Nouadhibou,MR,2022-09-06 04:42:28,20.9310,-17.0347,75.18,78,0,11.50,clear sky
2,2,Punta Arenas,CL,2022-09-06 04:42:28,-53.1500,-70.9167,33.91,86,40,20.71,light intensity shower rain
3,3,Hot Springs,US,2022-09-06 04:42:29,34.5037,-93.0552,72.72,90,100,3.00,overcast clouds
4,4,Saldanha,ZA,2022-09-06 04:42:29,-33.0117,17.9442,48.25,80,7,5.44,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Nouadhibou,MR,2022-09-06 04:42:28,20.9310,-17.0347,75.18,78,0,11.50,clear sky
6,6,Cayenne,GF,2022-09-06 04:42:30,4.9333,-52.3333,77.04,94,96,0.00,overcast clouds
7,7,Porto Novo,BJ,2022-09-06 04:42:30,6.4965,2.6036,75.02,93,51,4.90,broken clouds
9,9,Lae,PG,2022-09-06 04:42:31,-6.7221,146.9847,92.07,66,98,8.08,overcast clouds
13,13,Atuona,PF,2022-09-06 04:42:32,-9.8000,-139.0333,76.69,71,0,19.71,clear sky
14,14,Hilo,US,2022-09-06 04:42:32,19.7297,-155.0900,79.29,96,100,1.01,moderate rain
22,22,Gwadar,PK,2022-09-06 04:42:35,25.1216,62.3254,81.18,75,98,6.96,overcast clouds
31,31,Samarai,PG,2022-09-06 04:42:37,-10.6167,150.6667,82.94,73,1,11.14,clear sky
34,34,Shache,CN,2022-09-06 04:42:38,38.4167,77.2406,85.44,13,47,5.12,scattered clouds
36,36,Naze,JP,2022-09-06 04:42:38,28.3667,129.4833,83.32,81,100,7.74,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                243
City                   243
Country                243
Date                   243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df= preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nouadhibou,MR,75.18,clear sky,20.9310,-17.0347,
6,Cayenne,GF,77.04,overcast clouds,4.9333,-52.3333,
7,Porto Novo,BJ,75.02,broken clouds,6.4965,2.6036,
9,Lae,PG,92.07,overcast clouds,-6.7221,146.9847,
13,Atuona,PF,76.69,clear sky,-9.8000,-139.0333,
14,Hilo,US,79.29,moderate rain,19.7297,-155.0900,
22,Gwadar,PK,81.18,overcast clouds,25.1216,62.3254,
31,Samarai,PG,82.94,clear sky,-10.6167,150.6667,
34,Shache,CN,85.44,scattered clouds,38.4167,77.2406,
36,Naze,JP,83.32,light rain,28.3667,129.4833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!='']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nouadhibou,MR,75.18,clear sky,20.9310,-17.0347,El Medina
6,Cayenne,GF,77.04,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
7,Porto Novo,BJ,75.02,broken clouds,6.4965,2.6036,Tour Eiffel Hotel Benin
9,Lae,PG,92.07,overcast clouds,-6.7221,146.9847,Phils Hotel
13,Atuona,PF,76.69,clear sky,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
...,...,...,...,...,...,...,...
697,Yarada,IN,89.92,haze,17.6500,83.2667,Yarada Jungle Beach Resorts
698,Tual,ID,80.49,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
699,Acapulco,MX,77.34,overcast clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
700,Guiratinga,BR,75.25,broken clouds,-16.3494,-53.7583,Hotel Angra do Cerrado - Guiratinga MT


In [10]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
#<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))